[View in Colaboratory](https://colab.research.google.com/github/KushalVenkatesh/Simple_Linear_Regression_from_Scratch_using_Ordinary_Least_Squares_Method/blob/master/applying_a_neural_network_model.ipynb)

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib

In [0]:
import os
import keras

In [0]:
from google.colab import files
files.upload()

In [11]:
train = pd.read_csv("traindata.csv")
print('Shape of the train data with all features:', train.shape)
train = train.select_dtypes(exclude=['object'])
print("")
print('Shape of the train data with numerical features:', train.shape)
train.drop('Id',axis = 1, inplace = True)
train.fillna(0,inplace=True)

test = pd.read_csv("testdata.csv")
test = test.select_dtypes(exclude=['object'])
ID = test.Id
test.fillna(0,inplace=True)
test.drop('Id',axis = 1, inplace = True)

print("")
print("List of features contained our dataset:",list(train.columns))

Shape of the train data with all features: (1460, 81)

Shape of the train data with numerical features: (1460, 38)

List of features contained our dataset: ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']


In [36]:
train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,0.235294,0.207668,0.062802,0.625,0.428571,0.953488,0.883333,0.12250,0.416274,0.0,...,0.000000,0.167582,0.000000,0.0,0.0,0.0,0.0,0.090909,0.50,0.403057
1,0.000000,0.255591,0.072904,0.500,0.857143,0.744186,0.433333,0.00000,0.576651,0.0,...,0.404891,0.000000,0.000000,0.0,0.0,0.0,0.0,0.363636,0.25,0.340220
2,0.235294,0.217252,0.087396,0.625,0.428571,0.937984,0.866667,0.10125,0.286557,0.0,...,0.000000,0.115385,0.000000,0.0,0.0,0.0,0.0,0.727273,0.50,0.437966
3,0.294118,0.191693,0.072464,0.625,0.428571,0.271318,0.333333,0.00000,0.127358,0.0,...,0.000000,0.096154,0.824242,0.0,0.0,0.0,0.0,0.090909,0.00,0.243639
4,0.235294,0.268371,0.113835,0.750,0.428571,0.930233,0.833333,0.21875,0.386203,0.0,...,0.260870,0.230769,0.000000,0.0,0.0,0.0,0.0,1.000000,0.50,0.499638


In [37]:
train.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1314.000000,1314.000000,1314.000000,1314.000000,1314.000000,1314.000000,1314.000000,1314.000000,1314.000000,1314.000000,...,1314.000000,1314.000000,1314.000000,1314.000000,1314.000000,1314.000000,1314.000000,1314.000000,1314.000000,1314.000000
mean,0.204450,0.182605,0.074281,0.509228,0.509893,0.718683,0.581710,0.058519,0.249123,0.027987,...,0.123456,0.116422,0.058035,0.006956,0.033138,0.002023,0.001966,0.482358,0.448250,0.324588
std,0.237785,0.106167,0.048540,0.161356,0.150135,0.224552,0.344100,0.099709,0.239852,0.102357,...,0.163005,0.157305,0.167228,0.059462,0.128241,0.042528,0.028771,0.242530,0.332008,0.150661
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.137380,0.054458,0.375000,0.428571,0.573643,0.283333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.363636,0.250000,0.220366
50%,0.176471,0.201278,0.070506,0.500000,0.428571,0.736434,0.733333,0.000000,0.226415,0.000000,...,0.000000,0.065934,0.000000,0.000000,0.000000,0.000000,0.000000,0.454545,0.500000,0.293093
75%,0.294118,0.249201,0.087893,0.625000,0.571429,0.937984,0.900000,0.096094,0.409788,0.000000,...,0.228261,0.175824,0.000000,0.000000,0.000000,0.000000,0.000000,0.636364,0.750000,0.399566
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [38]:
test.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,0.000000,0.255591,0.090664,0.375,0.571429,0.627907,0.183333,0.0000,0.275943,0.097693,...,0.525180,0.190217,0.000000,0.0,0.0,0.311688,0.0,0.000000,0.454545,1.0
1,0.000000,0.258786,0.113896,0.500,0.571429,0.604651,0.133333,0.0675,0.544222,0.000000,...,0.224460,0.533967,0.098901,0.0,0.0,0.000000,0.0,0.806452,0.454545,1.0
2,0.235294,0.236422,0.110058,0.375,0.428571,0.906977,0.800000,0.0000,0.466392,0.000000,...,0.346763,0.288043,0.093407,0.0,0.0,0.000000,0.0,0.000000,0.181818,1.0
3,0.235294,0.249201,0.076224,0.500,0.571429,0.914729,0.800000,0.0125,0.354953,0.000000,...,0.338129,0.489130,0.098901,0.0,0.0,0.000000,0.0,0.000000,0.454545,1.0
4,0.588235,0.137380,0.032543,0.750,0.428571,0.868217,0.700000,0.0000,0.155071,0.000000,...,0.364029,0.000000,0.225275,0.0,0.0,0.374026,0.0,0.000000,0.000000,1.0


In [39]:
test.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,...,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,0.219873,0.185017,0.074829,0.509853,0.507686,0.708200,0.561046,0.062296,0.258787,0.035674,...,0.339888,0.126596,0.132731,0.073465,0.003532,0.044323,0.002364,0.003753,0.464016,0.442426
std,0.251452,0.103086,0.043527,0.179601,0.159106,0.235582,0.352174,0.110626,0.268430,0.119877,...,0.156350,0.173566,0.189240,0.203721,0.039779,0.147038,0.041317,0.040697,0.247494,0.325435
min,0.000000,0.000000,0.001493,-0.125000,-0.142857,-0.007752,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.140575,0.053501,0.375000,0.428571,0.565891,0.216667,0.000000,0.000000,0.000000,...,0.228417,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.272727,0.250000
50%,0.176471,0.201278,0.071138,0.500000,0.428571,0.720930,0.700000,0.000000,0.206368,0.000000,...,0.345324,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.454545,0.500000
75%,0.294118,0.249201,0.089746,0.625000,0.571429,0.937984,0.900000,0.101250,0.443396,0.000000,...,0.414388,0.228261,0.197802,0.000000,0.000000,0.000000,0.000000,0.000000,0.636364,0.750000
max,1.000000,0.638978,0.485731,1.000000,1.000000,1.007752,1.000000,0.806250,2.364387,1.035278,...,1.070504,1.934783,2.038462,3.066667,0.708661,1.496104,1.084011,1.096774,1.000000,1.000000


In [12]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest

clf = IsolationForest(max_samples = 100, random_state = 42)
clf.fit(train)
y_noano = clf.predict(train)
y_noano = pd.DataFrame(y_noano, columns = ['Top'])
y_noano[y_noano['Top'] == 1].index.values

train = train.iloc[y_noano[y_noano['Top'] == 1].index.values]
train.reset_index(drop = True, inplace = True)
print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
print("Number of rows without outliers:", train.shape[0])

Number of Outliers: 146
Number of rows without outliers: 1314


In [13]:
import warnings
warnings.filterwarnings('ignore')

col_train = list(train.columns)
col_train_bis = list(train.columns)

col_train_bis.remove('SalePrice')

mat_train = np.matrix(train)
mat_test  = np.matrix(test)
mat_new = np.matrix(train.drop('SalePrice',axis = 1))
mat_y = np.array(train.SalePrice).reshape((1314,1))

prepro_y = MinMaxScaler()
prepro_y.fit(mat_y)

prepro = MinMaxScaler()
prepro.fit(mat_train)

prepro_test = MinMaxScaler()
prepro_test.fit(mat_new)

train = pd.DataFrame(prepro.transform(mat_train),columns = col_train)
test  = pd.DataFrame(prepro_test.transform(mat_test),columns = col_train_bis)

train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,0.235294,0.207668,0.062802,0.625,0.428571,0.953488,0.883333,0.12250,0.416274,0.0,...,0.000000,0.167582,0.000000,0.0,0.0,0.0,0.0,0.090909,0.50,0.403057
1,0.000000,0.255591,0.072904,0.500,0.857143,0.744186,0.433333,0.00000,0.576651,0.0,...,0.404891,0.000000,0.000000,0.0,0.0,0.0,0.0,0.363636,0.25,0.340220
2,0.235294,0.217252,0.087396,0.625,0.428571,0.937984,0.866667,0.10125,0.286557,0.0,...,0.000000,0.115385,0.000000,0.0,0.0,0.0,0.0,0.727273,0.50,0.437966
3,0.294118,0.191693,0.072464,0.625,0.428571,0.271318,0.333333,0.00000,0.127358,0.0,...,0.000000,0.096154,0.824242,0.0,0.0,0.0,0.0,0.090909,0.00,0.243639
4,0.235294,0.268371,0.113835,0.750,0.428571,0.930233,0.833333,0.21875,0.386203,0.0,...,0.260870,0.230769,0.000000,0.0,0.0,0.0,0.0,1.000000,0.50,0.499638


In [0]:
# Listing the features
COLUMNS = col_train
FEATURES = col_train_bis
LABEL = "SalePrice"

# Columns
feature_cols = FEATURES

# Training set and Prediction set with the feature to predict
training_set = train[COLUMNS]
prediction_set = train.SalePrice

# Training and Testing
x_train, x_test, y_train, y_test = train_test_split(training_set[FEATURES] , prediction_set, test_size=0.33, random_state=42)
y_train = pd.DataFrame(y_train, columns = [LABEL])
training_set = pd.DataFrame(x_train, columns = FEATURES).merge(y_train, left_index = True, right_index = True)
training_set.head()

# Training
training_sub = training_set[col_train]

In [15]:
# Repeating the above for the test set
y_test = pd.DataFrame(y_test, columns = [LABEL])
testing_set = pd.DataFrame(x_test, columns = FEATURES).merge(y_test, left_index = True, right_index = True)
testing_set.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
1232,0.235294,0.239617,0.074221,0.625,0.571429,0.914729,0.800000,0.00000,0.574882,0.0,...,0.343750,0.467033,0.0,0.0,0.0,0.0,0.000000,0.454545,0.00,0.556191
76,0.352941,0.000000,0.063505,0.625,0.428571,0.891473,0.766667,0.01375,0.000000,0.0,...,0.163043,0.197802,0.0,0.0,0.0,0.0,0.045161,0.363636,0.75,0.309966
962,0.176471,0.159744,0.055205,0.375,0.142857,0.527132,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.636364,0.00,0.130767
433,0.588235,0.102236,0.028107,0.500,0.428571,0.914729,0.800000,0.07250,0.528892,0.0,...,0.000000,0.343407,0.0,0.0,0.0,0.0,0.000000,0.363636,0.00,0.299493
1110,0.823529,0.115016,0.011665,0.625,0.428571,0.953488,0.883333,0.06625,0.000000,0.0,...,0.000000,0.074176,0.0,0.0,0.0,0.0,0.000000,0.454545,1.00,0.326257


In [16]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

seed = 7
np.random.seed(seed)

# Model
model = Sequential()
model.add(Dense(200, input_dim=36, kernel_initializer='normal', activation='relu'))
model.add(Dense(100, kernel_initializer='normal', activation='relu'))
model.add(Dense(50, kernel_initializer='normal', activation='relu'))
model.add(Dense(25, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
# Compiling the model
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adadelta())

feature_cols = training_set[FEATURES]
labels = training_set[LABEL].values

model.fit(np.array(feature_cols), np.array(labels), epochs=100, batch_size=10)

Epoch 1/100
880/880 [==============================] - 0s 414us/step - loss: 0.0241
Epoch 2/100
880/880 [==============================] - 0s 182us/step - loss: 0.0057
Epoch 3/100
880/880 [==============================] - 0s 177us/step - loss: 0.0049
Epoch 4/100
880/880 [==============================] - 0s 173us/step - loss: 0.0041
Epoch 5/100
880/880 [==============================] - 0s 181us/step - loss: 0.0037
Epoch 6/100
880/880 [==============================] - 0s 196us/step - loss: 0.0039
Epoch 7/100
880/880 [==============================] - 0s 191us/step - loss: 0.0038
Epoch 8/100
880/880 [==============================] - 0s 185us/step - loss: 0.0035
Epoch 9/100
880/880 [==============================] - 0s 188us/step - loss: 0.0036
Epoch 10/100
880/880 [==============================] - 0s 191us/step - loss: 0.0032
Epoch 11/100
880/880 [==============================] - 0s 195us/step - loss: 0.0033
Epoch 12/100
880/880 [==============================] - 0s 190us/step - lo

Epoch 22/100
880/880 [==============================] - 0s 205us/step - loss: 0.0028
Epoch 23/100
880/880 [==============================] - 0s 209us/step - loss: 0.0030
Epoch 24/100
880/880 [==============================] - 0s 199us/step - loss: 0.0027
Epoch 25/100
880/880 [==============================] - 0s 213us/step - loss: 0.0027
Epoch 26/100
880/880 [==============================] - 0s 196us/step - loss: 0.0025
Epoch 27/100
880/880 [==============================] - 0s 204us/step - loss: 0.0027
Epoch 28/100
880/880 [==============================] - 0s 203us/step - loss: 0.0028
Epoch 29/100
880/880 [==============================] - 0s 211us/step - loss: 0.0026
Epoch 30/100
880/880 [==============================] - 0s 200us/step - loss: 0.0024
Epoch 31/100
880/880 [==============================] - 0s 203us/step - loss: 0.0027
Epoch 32/100
880/880 [==============================] - 0s 195us/step - loss: 0.0026
Epoch 33/100
880/880 [==============================] - 0s 199us/

880/880 [==============================] - 0s 208us/step - loss: 0.0024
Epoch 43/100
880/880 [==============================] - 0s 196us/step - loss: 0.0024
Epoch 44/100
880/880 [==============================] - 0s 205us/step - loss: 0.0024
Epoch 45/100
880/880 [==============================] - 0s 202us/step - loss: 0.0024
Epoch 46/100
880/880 [==============================] - 0s 198us/step - loss: 0.0021
Epoch 47/100
880/880 [==============================] - 0s 202us/step - loss: 0.0024
Epoch 48/100
880/880 [==============================] - 0s 208us/step - loss: 0.0026
Epoch 49/100
880/880 [==============================] - 0s 191us/step - loss: 0.0022
Epoch 50/100
880/880 [==============================] - 0s 219us/step - loss: 0.0021
Epoch 51/100
880/880 [==============================] - 0s 190us/step - loss: 0.0022
Epoch 52/100
880/880 [==============================] - 0s 199us/step - loss: 0.0024
Epoch 53/100
880/880 [==============================] - 0s 191us/step - loss: 

Epoch 63/100
880/880 [==============================] - 0s 221us/step - loss: 0.0022
Epoch 64/100
880/880 [==============================] - 0s 199us/step - loss: 0.0021
Epoch 65/100
880/880 [==============================] - 0s 219us/step - loss: 0.0025
Epoch 66/100
880/880 [==============================] - 0s 202us/step - loss: 0.0021
Epoch 67/100
880/880 [==============================] - 0s 213us/step - loss: 0.0019
Epoch 68/100
880/880 [==============================] - 0s 201us/step - loss: 0.0021
Epoch 69/100
880/880 [==============================] - 0s 217us/step - loss: 0.0022
Epoch 70/100
880/880 [==============================] - 0s 196us/step - loss: 0.0021
Epoch 71/100
880/880 [==============================] - 0s 207us/step - loss: 0.0021
Epoch 72/100
880/880 [==============================] - 0s 212us/step - loss: 0.0020
Epoch 73/100
880/880 [==============================] - 0s 199us/step - loss: 0.0021
Epoch 74/100
880/880 [==============================] - 0s 215us/

880/880 [==============================] - 0s 195us/step - loss: 0.0020
Epoch 84/100
880/880 [==============================] - 0s 189us/step - loss: 0.0019
Epoch 85/100
880/880 [==============================] - 0s 199us/step - loss: 0.0019
Epoch 86/100
880/880 [==============================] - 0s 210us/step - loss: 0.0021
Epoch 87/100
880/880 [==============================] - 0s 195us/step - loss: 0.0019
Epoch 88/100
880/880 [==============================] - 0s 203us/step - loss: 0.0019
Epoch 89/100
880/880 [==============================] - 0s 189us/step - loss: 0.0019
Epoch 90/100
880/880 [==============================] - 0s 204us/step - loss: 0.0020
Epoch 91/100
880/880 [==============================] - 0s 193us/step - loss: 0.0019
Epoch 92/100
880/880 [==============================] - 0s 199us/step - loss: 0.0018
Epoch 93/100
880/880 [==============================] - 0s 191us/step - loss: 0.0016
Epoch 94/100
880/880 [==============================] - 0s 192us/step - loss: 

In [17]:
# Evaluating the test set created by train_test_split
model.evaluate(np.array(feature_cols), np.array(labels))

880/880 [==============================] - 0s 74us/step


0.0017035445515913043

In [20]:
feature_cols = testing_set[FEATURES]
labels = testing_set[LABEL].values

model.fit(np.array(feature_cols), np.array(labels), epochs=100, batch_size=10)

Epoch 1/100
434/434 [==============================] - 0s 199us/step - loss: 0.0033
Epoch 2/100
434/434 [==============================] - 0s 180us/step - loss: 0.0025
Epoch 3/100
434/434 [==============================] - 0s 185us/step - loss: 0.0025
Epoch 4/100
434/434 [==============================] - 0s 190us/step - loss: 0.0020
Epoch 5/100
434/434 [==============================] - 0s 198us/step - loss: 0.0021
Epoch 6/100
434/434 [==============================] - 0s 205us/step - loss: 0.0021
Epoch 7/100
434/434 [==============================] - 0s 194us/step - loss: 0.0019
Epoch 8/100
434/434 [==============================] - 0s 197us/step - loss: 0.0017
Epoch 9/100
434/434 [==============================] - 0s 188us/step - loss: 0.0021
Epoch 10/100
434/434 [==============================] - 0s 186us/step - loss: 0.0019
Epoch 11/100
434/434 [==============================] - 0s 208us/step - loss: 0.0018
Epoch 12/100
434/434 [==============================] - 0s 181us/step - lo

434/434 [==============================] - 0s 201us/step - loss: 0.0016
Epoch 29/100
434/434 [==============================] - 0s 219us/step - loss: 0.0014
Epoch 30/100
434/434 [==============================] - 0s 194us/step - loss: 0.0015
Epoch 31/100
434/434 [==============================] - 0s 196us/step - loss: 0.0014
Epoch 32/100
434/434 [==============================] - 0s 184us/step - loss: 0.0012
Epoch 33/100
434/434 [==============================] - 0s 214us/step - loss: 0.0014
Epoch 34/100
434/434 [==============================] - 0s 207us/step - loss: 0.0015
Epoch 35/100
434/434 [==============================] - 0s 204us/step - loss: 0.0012
Epoch 36/100
434/434 [==============================] - 0s 202us/step - loss: 0.0013
Epoch 37/100
434/434 [==============================] - 0s 220us/step - loss: 0.0016
Epoch 38/100
434/434 [==============================] - 0s 203us/step - loss: 0.0014
Epoch 39/100
434/434 [==============================] - 0s 198us/step - loss: 

434/434 [==============================] - 0s 201us/step - loss: 9.5633e-04
Epoch 56/100
434/434 [==============================] - 0s 203us/step - loss: 0.0013
Epoch 57/100
434/434 [==============================] - 0s 208us/step - loss: 0.0010
Epoch 58/100
434/434 [==============================] - 0s 216us/step - loss: 0.0011
Epoch 59/100
434/434 [==============================] - 0s 214us/step - loss: 9.7332e-04
Epoch 60/100
434/434 [==============================] - 0s 215us/step - loss: 0.0012
Epoch 61/100
434/434 [==============================] - 0s 224us/step - loss: 0.0011
Epoch 62/100
434/434 [==============================] - 0s 208us/step - loss: 0.0011
Epoch 63/100
434/434 [==============================] - 0s 200us/step - loss: 9.4573e-04
Epoch 64/100
434/434 [==============================] - 0s 213us/step - loss: 0.0010
Epoch 65/100
434/434 [==============================] - 0s 214us/step - loss: 0.0011
Epoch 66/100
434/434 [==============================] - 0s 215us/s

434/434 [==============================] - 0s 203us/step - loss: 8.8353e-04
Epoch 82/100
434/434 [==============================] - 0s 216us/step - loss: 9.3572e-04
Epoch 83/100
434/434 [==============================] - 0s 229us/step - loss: 8.8128e-04
Epoch 84/100
434/434 [==============================] - 0s 235us/step - loss: 9.2169e-04
Epoch 85/100
434/434 [==============================] - 0s 214us/step - loss: 8.5610e-04
Epoch 86/100
434/434 [==============================] - 0s 224us/step - loss: 8.9183e-04
Epoch 87/100
434/434 [==============================] - 0s 214us/step - loss: 9.3098e-04
Epoch 88/100
434/434 [==============================] - 0s 207us/step - loss: 7.3309e-04
Epoch 89/100
434/434 [==============================] - 0s 210us/step - loss: 9.8153e-04
Epoch 90/100
434/434 [==============================] - 0s 227us/step - loss: 0.0012
Epoch 91/100
434/434 [==============================] - 0s 213us/step - loss: 8.9265e-04
Epoch 92/100
434/434 [================

In [21]:
# Evaluating the given test set
model.evaluate(np.array(feature_cols), np.array(labels))

434/434 [==============================] - 0s 39us/step


0.00047487423064247254